# BERT 기반 복합 토픽 모델 CTM 실습
* 코드 출처: [딥 러닝을 이용한 자연어 처리 입문 / 19-06 BERT 기반 복합 토픽 모델(Combined Topic Models, CTM)](https://wikidocs.net/161310)
* 데이터셋 출처: [Top 20 Play Store App Reviews (Daily Update)](https://www.kaggle.com/datasets/odins0n/top-20-play-store-app-reviews-daily-update?select=all_combined.csv)
    * 본 코드에서는 위 코드의 데이터셋을 사용하지 않고 Kaggle의 App Reviews 데이터셋을 이용하였습니다. 자세한 내용은 위 링크 참고해주세요.
* 본 코드는 아래와 같은 순서로 구성되어 있습니다.
    1. 모델 및 데이터셋 정의
    2. 데이터 로드 & 전처리
    3. 임베딩
    4. 모델 학습
    5. 결과 출력
* 주의사항: contextualized-topic-models 패키지가 설치되지 않은 경우, `pip install contextualized-topic-models==2.2.0`을 수행해주어야 하며 설치 속도가 (매우매우) 오래 걸림
---

## 01. 모델 및 데이터셋 정의
### 01-1. Model: CTM(Combined Topic Models)
* 문맥을 반영할 수 있는 BERT의 임베딩 방법에 기존 토픽 모델링의 비지도 학습 능력을 결합하여 문서의 주제를 가져오는 모델을 문맥을 반영한 토픽 모델(Contextualized Topic Models)이라고 하고, 그 중 하나가 **복합 토픽 모델**(Combined Topic Models, CTM)이다.

### 01-2. Dataset: Top 20 Play Store App Reviews (Daily Update)
* Facebook, WhatsApp을 포함하여 Google Play Store 상위 20개 앱의 리뷰가 들어있는 데이터셋이다.
* 데이터 개수는 앱당 1만개씩 포함하고 있다.


---
## 02. 데이터 로드 & 전처리

In [1]:
import pandas as pd
from contextualized_topic_models.models.ctm import CombinedTM
from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation
from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessing
import nltk

In [2]:
# 데이터 로드 (앱 리뷰 파일)
data = pd.read_csv('./Dataset/all_combined.csv')
print('데이터 총 개수:', len(data))
print('데이터 5개 미리 보기')
data[:5]

데이터 총 개수: 200000
데이터 5개 미리 보기


,reviewId,content,score,app
0,3378decf-f94f-4275-b39e-0b6f37dab0e0,Entertainment,4,Facebook
1,7233651d-4343-4f3e-aa10-b422741eaa5a,Non secure platform.My Facebook id automatic f...,1,Facebook
2,706d5c13-5dea-4fbd-813b-8f90dd7af626,Pleas donlowad ho ja,5,Facebook
3,90ac91c4-41f3-41d0-b932-188b431893e4,ुिंपबवयू,4,Facebook
4,3a01939b-0996-4602-b9cd-1fa478d5d5b3,I'm having trouble with it shutting and I can'...,1,Facebook


In [3]:
# 앱의 종류와 score로 그룹핑하여 텍스트 합치기
app_list = data['app'].unique().tolist()
score_list = sorted(data['score'].unique().tolist())

title_list = [(app_name, score) for app_name in app_list for score in  score_list]
documents  = []


for i, (app_name, score) in enumerate(title_list):
    for score in score_list:
        # 텍스트 연결
        contents = data[(data['score'] == score) & (data['app'] == app_name)]['content']
        contents = ' '.join(map(str, contents))
        
        # 딕셔너리 키로 집합 형태의 (앱 이름, 점수)를 사용
        # 딕셔너리 밸류로 포함되는 문자열을 모두 연결한 문자열을 사용
        documents .append(contents)
        
# 값이 잘 들어갔는지 확인
documents[:5]

['Non secure platform.My Facebook id automatic following people and page\'s I\'m having trouble with it shutting and I can\'t load any of my emails. Getting awfully fed up!! 😻 Not me Tell the setting for the reel, tell the setting to block whatever reel comes in your fb id and hide it from the timeline. Full glitch app Lalitakharadi Encourages fraud... There is unexpected error while i login my account fix this error kindly this is showing unable to login even my entered details are 1000% correct Total ads scam froud company Bad app Nice pàrformenc Ok Hate how your adds gets spammed in my game The amount of racists, homophobes, transphobes, fatphobes, etc I see on this app is ridiculous. Can\'t get app to open super slow We cannot gain our account access,when we accidentally lose our account. Every third post is an advertisement or suggested post... so sick of it taking up space... And what\'s with the random difficulties exiting the app after you\'re done??? Is this deliberate to make

* 아래 코드에서 ```AttributeError: 'CountVectorizer' object has no attribute 'get_feature_names'``` 이러한 에러 코드가 뜬다면, get_features_names가 get_features_out으로 변경되었기 때문이다.
* CTM 라이브러리의 data_preparation.py, preprocessing.py에 작성된 get_features_names를 **get_features_names_out**으로 변경해주어야 한다.

In [4]:
# 전처리: 불용어 및 특수문자 제거, 영단어의 소문자화 진행
sp = WhiteSpacePreprocessing(documents, stopwords_language='english')
preprocessed_documents, unpreprocessed_corpus, vocab = sp.preprocess()

print('전처리 전:')
print(unpreprocessed_corpus[:2])
print('전처리 후:')
print(preprocessed_documents[:2])

전처리 전:
['Non secure platform.My Facebook id automatic following people and page\'s I\'m having trouble with it shutting and I can\'t load any of my emails. Getting awfully fed up!! 😻 Not me Tell the setting for the reel, tell the setting to block whatever reel comes in your fb id and hide it from the timeline. Full glitch app Lalitakharadi Encourages fraud... There is unexpected error while i login my account fix this error kindly this is showing unable to login even my entered details are 1000% correct Total ads scam froud company Bad app Nice pàrformenc Ok Hate how your adds gets spammed in my game The amount of racists, homophobes, transphobes, fatphobes, etc I see on this app is ridiculous. Can\'t get app to open super slow We cannot gain our account access,when we accidentally lose our account. Every third post is an advertisement or suggested post... so sick of it taking up space... And what\'s with the random difficulties exiting the app after you\'re done??? Is this deliberate 

* 전처리 전/후 문서 모두를 사용하여 임베딩을 수행한다. 따라서 두 변수 모두 저장하여야 한다.
    1. 전처리 후 문서에서는 단어 집합(bag of words)을 얻는다.
    2. 1에서 얻은 단어 집합을 이용하여 전처리 전 문서(원본 문서)의 BERT 임베딩을 얻는다.

In [5]:
# 전체 단어 집합의 크기
# WhiteSpacePreprocessing의 기본 단어 집합 크기가 2000이므로 2000이 출력됨
# 수정하고 싶은 경우 WhiteSpacePreprocessing의 vocabulary_size를 수정

print('bag of words에 사용 될 단어 집합의 크기 :',len(vocab))

bag of words에 사용 될 단어 집합의 크기 : 2000


## 03. 임베딩 (Embedding)

In [6]:
# pretrained BERT를 불러와 임베딩을 수행한다
tp = TopicModelDataPreparation("paraphrase-distilroberta-base-v1")
training_dataset = tp.fit(text_for_contextual=unpreprocessed_corpus, text_for_bow=preprocessed_documents)

# 단어 집합의 개수를 출력했을 때, 이전에 구했던 단어 집합과 동일한 것을 알 수 있음.
# (그런데 나는 왜 1999가 나올까요...? ㅠㅠ)
len(tp.vocab)

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

1999

## 04. CTM 학습하기

In [7]:
# 토픽의 개수(n_componenets)는 50개로 선정
ctm = CombinedTM(bow_size=len(tp.vocab), contextual_size=768, n_components=50, num_epochs=20)
ctm.fit(training_dataset)

0it [00:00, ?it/s]

KeyboardInterrupt: 

## 05. 결과 출력

In [ ]:
# 토픽마다 상위 n개 보여주기
top5_keywords = ctm.get_topic_lists(5)
top5_keywords

In [ ]:
# 문서 예시: 3개의 문서에 대한 문서 내용과 키워드 출력
for i in range(3):
    print('문서내용:')
    print(unpreprocessed_corpus[i])
    print('도출된 키워드')
    print(top5_keywords[i])